In [1]:
import pandas as pd
from scipy.interpolate import interp1d

In [2]:
RAW_DATASET_DIR = 'Raw datasets/'
PROCESSED_OUTPUT_DIR = 'Processed Datasets/'

In [3]:
data = pd.read_csv(RAW_DATASET_DIR + "interest.csv")
MONTH_ABBREVIATION_TO_NUMERIC = {
    "JAN": "01",
    "FEB": "02",
    "MAR": "03",
    "APR": "04",
    "MAY": "05",
    "JUN": "06",
    "JUL": "07",
    "AUG": "08",
    "SEP": "09",
    "OCT": "10",
    "NOV": "11",
    "DEC": "12",
    "AVE":"NA",
}
data['Year'] = data['Year'].astype(str)
data['Month'] = data['Month'].map(MONTH_ABBREVIATION_TO_NUMERIC)
data['Date'] = data['Year'] + "-" + data['Month']
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m')
print(data)

   Unnamed: 0  Year Month  Interest_rate       Date
0           0  2018    10           4.50 2018-10-01
1           1  2019    04           4.75 2019-04-01
2           2  2019    08           4.25 2019-08-01
3           3  2020    01           4.00 2020-01-01
4           4  2020    04           2.75 2020-04-01
5           5  2021    01           2.00 2021-01-01
6           6  2022    04           2.00 2022-04-01
7           7  2022    07           3.25 2022-07-01
8           8  2023    02           6.00 2023-02-01
9           9  2024    10           6.00 2024-10-01


In [4]:
data.set_index('Date', inplace=True)

# Reindex to have all months between the minimum and maximum dates
date_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='MS')
data_full = data.reindex(date_range)

# Interpolate missing 'Rate' values
data_full['Interest_rate'] = data_full['Interest_rate'].interpolate()

# Reset index to include the full date range and optional to re-create Year/Month columns
data_full = data_full.reset_index().rename(columns={'index': 'Date'})
data_full['Year'] = data_full['Date'].dt.year
data_full['Month'] = data_full['Date'].dt.month

data_full = data_full.loc[:, ~data_full.columns.str.contains('^Unnamed')]
data_full.to_csv(PROCESSED_OUTPUT_DIR + "interest.csv")
